In [ ]:
#cu101 
!pip install torch-geometric \
  torch-sparse==latest+cu101 \
  torch-scatter==latest+cu101 \
  torch-cluster==latest+cu101 \
  -f https://pytorch-geometric.com/whl/torch-1.4.0.html

In [ ]:
import networkx as nx
from sklearn.model_selection import train_test_split
import numpy as np 
import torch
from matplotlib import pyplot as plt 
#from torch_geometric.datasets import Entities 
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader
from torch_geometric.nn import GINConv, global_add_pool, GCNConv, global_mean_pool
from torch_geometric.utils import convert

import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU
from torch_geometric.data import DataLoader

In [ ]:
class dataset_loading:
    def __init__(self):
        pass 
    #SBM generator
    def generate_SBM(self,Graphs_num=300,nodes_per_graph=60,block_size=10,fraction=0.3,mult_factor=2,avg_deg=10,test_size=0.2):
        blocks_num=int(nodes_per_graph/block_size)
        sizes=[block_size]*blocks_num
        G,y=[],[]
        for i in range (Graphs_num):                  
            p_in=fraction  if i <Graphs_num/2 else fraction*mult_factor
            p_out=(avg_deg-(block_size-1)*p_in)/(nodes_per_graph-block_size)
            p=p_out*np.ones([blocks_num]*2)+(p_in-p_out)*np.eye(blocks_num)
            #print(p_in,p_out)
            G.append(nx.stochastic_block_model(sizes, p))
            y.append(0 if i<Graphs_num/2 else 1)            
        G_train, G_test, y_train, y_test = train_test_split(G, y, test_size=test_size)
        return (G_train,y_train),(G_test,y_test)

mult_factor=2 # here the code is not prepared yet to handle a vector of values as in the main code
(Gtr,ytr),(Gts,yts)=dataset_loading().generate_SBM(mult_factor=mult_factor)     
#transforming the dataset to Torch_geometry framework 
for i,g in enumerate (Gtr):
  dg=list(g.degree(g.nodes))
  dg=dict([(i,[j]) for (i,j) in dg])
  nx.set_node_attributes(g,dg,'x')
Gtr=[convert.from_networkx(g) for g in Gtr]
for i,g in enumerate (Gtr): g.y=torch.tensor([ytr[i]],dtype=torch.long)

for i,g in enumerate (Gts):
  dg=list(g.degree(g.nodes))
  dg=dict([(i,[j]) for (i,j) in dg])
  nx.set_node_attributes(g,dg,'x')
Gts=[convert.from_networkx(g) for g in Gts]
for i,g in enumerate (Gts): g.y=torch.tensor([yts[i]], dtype=torch.long)

test_dataset = Gts                   
train_dataset = Gtr
test_loader = DataLoader(test_dataset, batch_size=128)
train_loader = DataLoader(train_dataset, batch_size=128)

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        num_features=1
        dim = 32
        hidden=dim
        num_layers=4
        num_classes=2
        self.conv1 = GCNConv( num_features, hidden)
        self.bn1 = torch.nn.BatchNorm1d(hidden)                  
        self.convs = torch.nn.ModuleList()
        self.bn=torch.nn.ModuleList()
        for i in range(num_layers - 1):
            self.convs.append(GCNConv(hidden, hidden))
            self.bn.append( torch.nn.BatchNorm1d(hidden) )
        self.lin1 = Linear(hidden, hidden)
        self.lin2 = Linear(hidden,  num_classes)
        self.bn1 = torch.nn.BatchNorm1d(dim)                  #######################################################


    def forward(self, x, edge_index, batch):
        #x, edge_index, batch = data.x, data.edge_index, data.batch          #############################################3
        x = F.relu(self.conv1(x.float(), edge_index))
        x=self.bn1(x)
        for i,conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            x=self.bn[i](x)
        x = global_mean_pool(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.1, training=self.training)     #######################################################################3
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)




device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


def train(epoch):
    model.train()
    
    if epoch == 51:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.1 * param_group['lr']
    
    for param_group in optimizer.param_groups: pass
        #param_group['lr'] =  param_group['lr']/(1+0.1*epoch)
    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        #print(data.x)
        optimizer.zero_grad()
        output = model(data.x, data.edge_index, data.batch)
        loss = F.nll_loss(output, data.y)
        loss.backward()
        loss_all += loss.item() * data.num_graphs
        optimizer.step()
    return loss_all / len(train_dataset)


def test(loader):
    model.eval()

    correct = 0
    for data in loader:
        data = data.to(device)
        output = model(data.x, data.edge_index, data.batch)
        pred = output.max(dim=1)[1]
        correct += pred.eq(data.y).sum().item()
    return correct / len(loader.dataset)


for epoch in range(1, 1000):
    train_loss = train(epoch)
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    #test_acc = 0
    print('Epoch: {:03d}, Train Loss: {:.7f}, '
          'Train Acc: {:.1f}%, Test Acc: {:.0f}%'.format(epoch, train_loss,
                                                       100*train_acc, 100*test_acc))

In [ ]:
# do not read this cell, it is unimportant. I just tried some things. 

a=TUDataset(root=".", name="MUTAG").shuffle()
print(len(a))
print(a[0].keys)
print(a[0].y)
print(a[100].y)
a=convert.to_networkx(a[0],node_attrs='x')
print()
color = nx.get_node_attributes(a, 'x')
print(a.nodes[0]['x'])
#print(a.nodes(data=True))
a=convert.from_networkx(a)
print(type(a))
print(a.y)
a.y=1
print(a.y)



#nx.draw(a)
#plt. show()
dataset = TUDataset('.', name='MUTAG').shuffle()
test_dataset = dataset[:1]
train_dataset = dataset[len(dataset) // 10:]
test_loader = DataLoader(test_dataset, batch_size=1)
train_loader = DataLoader(train_dataset, batch_size=128)


print(type(test_dataset))
print(dataset[0].y)
print(type(dataset[0]))
for data in test_loader:
  data = data.to(device)
  print(data.x.dtype)
print(dataset.num_features)